In [1]:
import pandas as pd
import numpy as np
import re

data = pd.read_csv('../data/train.csv')

## First step: tokenized data

Convert data to lowercase and remove all URLs

In [48]:
pd.options.mode.chained_assignment = None # silence warning related to overwriting original frame

tokenized_data = data.copy()
tokenized_data['text'] = tokenized_data['text'].str.lower()
tokenized_data['text'] = tokenized_data['text'].replace(to_replace = r'https?:\/\/.*[\r\n]*|[^\w\s]', value = " ", regex=True)

for i in range(len(tokenized_data['text'])):
    tokenized_data['text'][i] = [data for data in tokenized_data['text'][i].split(" ") if data]

## Second step: ranked data

Transform tokens into rank indicators. Low frequency words = high rank

1. Create a frequencies dictionary mapping each token to its frequency

In [ ]:
frequencies = {} # token : frequency

for tweet in tokenized_data['text']:
    for token in tweet:
        token = token.strip()    # strip necessary to remove whitespaces (newlines)
        if token:
            frequencies[token] = frequencies.get(token,0) + 1

2. Map each frequency to a rank

In [54]:
frequency_set = sorted(set(list(frequencies.values())))[::1]
num_ranks = len(frequency_set)
frequency_to_rank = {} # frequency : rank

for i, freq in enumerate(frequency_set):
    frequency_to_rank[freq] = i+1

,id,keyword,location,text,target
0,1,NaN,NaN,"[94, 2, 169, 190, 19, 186, 171, 41, 84, 9, 2, ...",1
1,4,NaN,NaN,"[63, 146, 55, 22, 1, 1, 10]",1
2,5,NaN,NaN,"[152, 8, 9, 187, 7, 188, 24, 169, 86, 1, 173, ...",1
3,6,NaN,NaN,"[21, 4, 133, 2, 9, 48, 11, 188, 109]",1
4,7,NaN,NaN,"[161, 105, 12, 171, 41, 170, 1, 8, 165, 48, 17...",1
...,...,...,...,...,...
7608,10869,NaN,NaN,"[99, 20, 12, 16, 189, 40, 74, 126, 27, 60]",1
7609,10870,NaN,NaN,"[2, 2, 190, 153, 186, 23, 49, 97, 188, 109, 72...",1
7610,10871,NaN,NaN,"[5, 6, 26, 14, 10, 10, 181, 186, 27, 10]",1
7611,10872,NaN,NaN,"[119, 15, 150, 148, 27, 6, 42, 175, 189, 88, 1...",1


3. Create ranked data

In [ ]:
ranked_data = tokenized_data.copy()
for i in range(len(ranked_data['text'])):
    ranked_data['text'][i] = [frequency_to_rank[frequencies[data.strip()]] for data in ranked_data['text'][i] if data.strip()]

display(ranked_data)

## Third step: padded data

Pad data with zeroes to ensure all vectors are of the same length

In [73]:
VECTOR_SIZE = 50
max_tweet_len = ranked_data['text'].str.len().max()
assert(VECTOR_SIZE >= max_tweet_len)

padded_data = ranked_data.copy()

for i in range(len(padded_data['text'])):
    tweet_len = len(padded_data['text'][i])
    num_zeros = VECTOR_SIZE - tweet_len
    assert(num_zeros > 0)
    padded_data['text'][i] = [0] * num_zeros + padded_data['text'][i]
    assert(len(padded_data['text'][i]) == VECTOR_SIZE)
display(padded_data)

,id,keyword,location,text,target
0,1,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,4,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,5,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,6,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,7,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...
7608,10869,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
7609,10870,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
7610,10871,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
7611,10872,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
